In [52]:
import tensorflow as tf
print('GPU',len(tf.config.list_physical_devices('GPU')))

GPU 1


In [53]:
labels_df=pd.read_csv('/kaggle/input/doctors-handwritten-prescription-bd-dataset/Doctor’s Handwritten Prescription BD dataset/Training/training_labels.csv')

In [54]:
print(labels_df.columns)

Index(['IMAGE', 'MEDICINE_NAME', 'GENERIC_NAME'], dtype='object')


In [55]:
import os
import tensorflow as tf
import pandas as pd
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam


In [56]:
image_folder = '/kaggle/input/doctors-handwritten-prescription-bd-dataset/Doctor’s Handwritten Prescription BD dataset/Training/training_words'
label_file = '/kaggle/input/doctors-handwritten-prescription-bd-dataset/Doctor’s Handwritten Prescription BD dataset/Training/training_labels.csv'

In [57]:
images=[]
labels=[]

In [58]:
filename_col='IMAGE'
label_col='MEDICINE_NAME'

In [59]:
labels_df.head()

,IMAGE,MEDICINE_NAME,GENERIC_NAME
0,0.png,Aceta,Paracetamol
1,1.png,Aceta,Paracetamol
2,2.png,Aceta,Paracetamol
3,3.png,Aceta,Paracetamol
4,4.png,Aceta,Paracetamol


In [60]:
labels_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3120 entries, 0 to 3119
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   IMAGE          3120 non-null   object
 1   MEDICINE_NAME  3120 non-null   object
 2   GENERIC_NAME   3120 non-null   object
dtypes: object(3)
memory usage: 73.2+ KB


In [61]:
if filename_col not in labels_df.columns or label_col not in labels_df.columns:
    raise ValueError(f"Specified columns '{filename_col}' or '{label_col}' not found in the CSV file.")

In [62]:
for index, row in labels_df.iterrows():
    img_path = os.path.join(image_folder, row[filename_col])
    img = load_img(img_path, target_size=(64, 64))  # Resize to a fixed size
    img = img_to_array(img)
    images.append(img)
    labels.append(row[label_col])

In [63]:
images = np.array(images, dtype="float") / 255.0  # Normalize pixel values
labels = np.array(labels)

In [64]:
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
labels_categorical = to_categorical(labels_encoded, num_classes=78)

In [65]:
X_train, X_test, y_train, y_test = train_test_split(images, labels_categorical, test_size=0.2, random_state=42)

In [66]:
model=Sequential()

In [67]:
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(78, activation='softmax')) 

/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [74]:
from keras.optimizers import Adam
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])


In [75]:
history=model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=20,batch_size=32)

Epoch 1/20


2024-06-15 13:14:48.729927: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 0: 6.39927, expected 5.58886
2024-06-15 13:14:48.729982: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 1: 5.92185, expected 5.11143
2024-06-15 13:14:48.729996: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 2: 5.99669, expected 5.18628
2024-06-15 13:14:48.730009: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 3: 6.48891, expected 5.6785
2024-06-15 13:14:48.730023: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 5: 6.18882, expected 5.37841
2024-06-15 13:14:48.730033: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 6: 6.76108, expected 5.95067
2024-06-15 13:14:48.730044: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 7: 6.21291, expected 5.4025
2024-06-15 13:14:48.730053: E external/local_xla/xla/serv

38/78 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0085 - loss: 4.4274 

I0000 00:00:1718457291.639748    1264 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1718457291.659988    1264 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


76/78 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0110 - loss: 4.4036

W0000 00:00:1718457292.757103    1267 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
2024-06-15 13:14:52.887933: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 23106: 6.38351, expected 5.64383
2024-06-15 13:14:52.887989: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 23194: 5.12334, expected 4.38367
2024-06-15 13:14:52.887998: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 23218: 6.35016, expected 5.61048
2024-06-15 13:14:52.888006: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 23240: 5.82993, expected 5.09026
2024-06-15 13:14:52.888013: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 23251: 6.18931, expected 5.44964
2024-06-15 13:14:52.888021: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 23270: 6.36522, expected 5.62554
2024-06-15 13:14:52.888029: E external/lo

78/78 ━━━━━━━━━━━━━━━━━━━━ 8s 24ms/step - accuracy: 0.0111 - loss: 4.4024 - val_accuracy: 0.0144 - val_loss: 4.3305
Epoch 2/20


W0000 00:00:1718457293.521002    1267 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


78/78 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.0257 - loss: 4.2963 - val_accuracy: 0.0593 - val_loss: 4.1340
Epoch 3/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0905 - loss: 3.9569 - val_accuracy: 0.2035 - val_loss: 3.4790
Epoch 4/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2148 - loss: 3.2318 - val_accuracy: 0.4006 - val_loss: 2.4110
Epoch 5/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4672 - loss: 2.0495 - val_accuracy: 0.5465 - val_loss: 1.8177
Epoch 6/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6266 - loss: 1.3759 - val_accuracy: 0.6138 - val_loss: 1.5332
Epoch 7/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7322 - loss: 0.9266 - val_accuracy: 0.6891 - val_loss: 1.3285
Epoch 8/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8290 - loss: 0.5971 - val_accuracy: 0.6811 - val_loss: 1.3157
Epoch 9/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8819 - loss: 0.3991 - val_accuracy: 0.6795 - val_loss: 1.4169
Epo

In [77]:
loss,accuracy=model.evaluate(X_test,y_test)

20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7292 - loss: 1.5056 


In [79]:
print("Accuracy",accuracy*100)

Accuracy 72.11538553237915
